In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
from PIL import Image
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.models as models

from COCODataset import MaskedCOCODataset, COCODataset
from COCOWrapper import COCOWrapper
from ImageLoader import ImageLoader
from ModelWrapper import ModelWrapper
from Train import train_model


In [3]:
model_choice = 1

root = '/home/gregory/Datasets/COCO/'
year = '2017'

save_dir = './DataAugmentation'
run_create = False

run_transfer_altered = True

In [4]:
# coco is a COCOWrapper, loader is an ImageLoader
def create(model_class, labeler_classes, coco, loader, save_dir = './DataAugmentation/'):
    
    save_location = '{}/{}{}-{}-{}/'.format(save_dir, coco.mode, coco.year, model_class, labeler_classes).replace("'", '').replace(" ", '')

    os.system('rm -rf {}'.format(save_location))
    os.system('mkdir {}'.format(save_location))
     
    # Start by getting all of the images that contain the target classes according to the labeler
    relevant_classes = [model_class]
    for c in labeler_classes:
        relevant_classes.append(c)
    
    img_objs = coco.get_images_with_cats(relevant_classes)
    n = len(img_objs)
    
    labels = []
    labels_masked = coco.get_cat_ids(labeler_classes)
    for i in range(n):
        img_obj = img_objs[i]
        file = '{}{}'.format(save_location, img_obj['file_name'])
     
        # Load and save the masked version of the image
        img = loader.load_img(img_obj, transform_apply = False, mask_apply = True, mask_classes = labeler_classes)
        img.save(file)
        
        # Get the label of the image
        annotations = coco.get_annotations(img_obj)
        label = np.zeros((91), dtype = np.float32)
        for ann in annotations:
            label[ann['category_id']] = 1.0
                
        # Remove the masked objects from the label
        label[labels_masked] = 0.0

        labels.append((file, label))
    
    with open('{}labels.p'.format(save_location), 'wb') as f:
        pickle.dump(labels, f)


In [5]:

if model_choice == 0:
    model = models.resnet18(pretrained = True)
    model.fc = torch.nn.Linear(in_features = 512, out_features = 91)
elif model_choice == 1:
    model = models.mobilenet_v2(pretrained = True)
    model.classifier[1] = torch.nn.Linear(in_features = 1280, out_features = 91)  
    
model.load_state_dict(torch.load('./model_transfer.pt'))
model.eval()
model.cuda()

wrapper = ModelWrapper(model) # Note:  the wrapper does not copy the model, so updating the model weights updates the wrapper

In [6]:

def my_dataloader(dataset):
    return torch.utils.data.DataLoader(dataset, batch_size = 32, shuffle = True, num_workers = 4)

model_class = 'skis'
labeler_classes = ['person']

# Unaltered Validation Set
dataset_original = COCODataset(root = root, mode = 'val', year = year)
dataloader_original = my_dataloader(dataset_original)

# Altered Validation Set
save_location = '{}/{}{}-{}-{}/'.format(save_dir, 'val', year, model_class, labeler_classes).replace("'", '').replace(" ", '')
dataset_altered = MaskedCOCODataset('{}labels.p'.format(save_location))
dataloader_altered = my_dataloader(dataset_altered)

# Augmented Train/Validation Sets
cocos = {}
loaders = {}
datasets = {}

for mode in ['val', 'train']:

    base = '{}{}{}/'.format(root, mode, year)

    coco = COCOWrapper(root = root, mode = mode, year = year)
    cocos[mode] = coco

    loader = ImageLoader(root = base, coco = coco.coco)
    loaders[mode] = loader
    
    save_location = '{}/{}{}-{}-{}/'.format(save_dir, coco.mode, coco.year, model_class, labeler_classes).replace("'", '').replace(" ", '')

    if run_create:
        create(model_class, labeler_classes, coco, loader)

    datasets[mode] = COCODataset(root = root, mode = mode, year = year, sources = ['{}labels.p'.format(save_location)])

dataloaders = {x: my_dataloader(datasets[x]) for x in ['train', 'val']}
dataset_sizes = {x: len(datasets[x]) for x in ['train', 'val']}

loading annotations into memory...
Done (t=0.31s)
creating index...
index created!
loading annotations into memory...
Done (t=0.31s)
creating index...
index created!
loading annotations into memory...
Done (t=0.45s)
creating index...
index created!
loading annotations into memory...
Done (t=9.93s)
creating index...
index created!
loading annotations into memory...
Done (t=11.17s)
creating index...
index created!


In [7]:

index = cocos['val'].get_cat_ids(model_class)[0]

print("Original Model")

print("Original Validation Set")

y_hat, y_true = wrapper.predict_dataset(dataloader_original)
precision, recall = wrapper.metrics(y_hat, y_true)

print("Precision ", precision[index])
print("Recall ", recall[index])

print("Altered Validation Set")

y_hat, y_true = wrapper.predict_dataset(dataloader_altered)
precision, recall = wrapper.metrics(y_hat, y_true)

print("Precision ", precision[index])
print("Recall ", recall[index])


Original Model
Original Validation Set
Precision  0.8333333333333334
Recall  0.75
Altered Validation Set
Precision  1.0
Recall  0.11666666666666667


In [8]:


model.train()

for param in model.parameters():
    param.requires_grad = False

if model_choice == 1:
    for param in model.classifier.parameters():
        param.requires_grad = True
        
    optim_params = model.classifier.parameters()
    
if run_transfer_altered:

    criterion = torch.nn.BCEWithLogitsLoss()

    optimizer = optim.Adam(optim_params, lr = 0.001)

    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size = 2, gamma = 0.1)

    model = train_model(model, dataloaders, dataset_sizes, criterion, optimizer, exp_lr_scheduler, num_epochs = 5)

    torch.save(model.state_dict(), './model_transfer_augented.pt')
    
else:
    model.load_state_dict(torch.load('./model_transfer_augented.pt'))
    
model.eval()


  0%|          | 0/3792 [00:00<?, ?it/s]

Epoch 0/4
----------


  0%|          | 0/160 [00:00<?, ?it/s]

train Loss: 0.0699 Acc: 0.9683


  0%|          | 0/3792 [00:00<?, ?it/s]

...

  0%|          | 0/160 [00:00<?, ?it/s]

train Loss: 0.0653 Acc: 0.9686


100%|██████████| 160/160 [00:10<00:00, 14.67it/s]

val Loss: 0.0635 Acc: 0.9688

Training complete in 21m 14s
Best val Acc: 0.968750


MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [9]:
print("Adjusted Model")

print("Original Validation Set")

y_hat, y_true = wrapper.predict_dataset(dataloader_original)
precision, recall = wrapper.metrics(y_hat, y_true)

print("Precision ", precision[index])
print("Recall ", recall[index])

print("Altered Validation Set")

y_hat, y_true = wrapper.predict_dataset(dataloader_altered)
precision, recall = wrapper.metrics(y_hat, y_true)

print("Precision ", precision[index])
print("Recall ", recall[index])


y_hat, y_true = wrapper.predict_dataset(dataloader_original)
precision, recall = wrapper.metrics(y_hat, y_true)

cocos['val'].show_metrics(precision, recall)

Adjusted Model
Original Validation Set
Precision  0.826530612244898
Recall  0.675
Altered Validation Set
Precision  1.0
Recall  0.9166666666666666

Object Precision Recall
person 0.8337391304347826 0.8900854066097289
bicycle 0.7333333333333333 0.1476510067114094
car 0.6739811912225705 0.40186915887850466
motorcycle 0.8 0.5031446540880503
airplane 0.8405797101449275 0.5979381443298969
bus 0.839622641509434 0.4708994708994709
train 0.8114754098360656 0.6305732484076433
truck 0.6862745098039216 0.14
boat 0.75 0.39669421487603307
traffic light 0.972972972972973 0.18848167539267016
fire hydrant 0.7878787878787878 0.3023255813953488
stop sign 0.7692307692307693 0.43478260869565216
parking meter 0.5714285714285714 0.21621621621621623
bench 0.8333333333333334 0.06382978723404255
bird 0.7321428571428571 0.328
cat 0.8194444444444444 0.6413043478260869
dog 0.8225806451612904 0.288135593220339
horse 0.725 0.453125
sheep 0.92 0.35384615384615387
cow 0.9615384615384616 0.28735632183908044
elephant 0